Ibrahim Johar Farooqi

23K-0074

Lab 7 Tasks

Task 1

In [ ]:
import copy
import math

#constants
EMPTY = '.'
WHITE = 'W'
WHITE_KING = 'WK'
BLACK = 'B'
BLACK_KING = 'BK'
ROWS, COLS = 8, 8

#dir: (dr, dc)
DIRECTIONS = {
    WHITE: [(-1, -1), (-1, 1)],
    BLACK: [(1, -1), (1, 1)],
    WHITE_KING: [(-1, -1), (-1, 1), (1, -1), (1, 1)],
    BLACK_KING: [(-1, -1), (-1, 1), (1, -1), (1, 1)],
}

class Move:
    def __init__(self, path, captures=None):
        self.path = path          #list of positions [(r,c), ..]
        self.src = path[0]
        self.dest = path[-1]
        self.captures = captures or []  #list of positions captured
    def __repr__(self):
        cap = f" captures {self.captures}" if self.captures else ""
        return f"move({self.src}->{self.dest}{cap})"

class CheckersGame:
    def __init__(self, depth=4):
        self.board = self._init_board()
        self.depth = depth
        self.turn = WHITE

    def _init_board(self):
        b = [[EMPTY]*COLS for _ in range(ROWS)]
        for r in range(ROWS):
            for c in range(COLS):
                if (r + c) % 2:
                    if r < 3:
                        b[r][c] = BLACK
                    elif r > 4:
                        b[r][c] = WHITE
        return b

    def print_board(self):
        print("  " + " ".join(map(str, range(COLS))))
        for r, row in enumerate(self.board):
            print(r, " ".join(cell.rjust(2) for cell in row))
        print()

    def inside(self, r, c):
        return 0 <= r < ROWS and 0 <= c < COLS

    def get_moves(self, player):
        # mandatory captures
        caps = []
        for r in range(ROWS):
            for c in range(COLS):
                if self.board[r][c] in (player, player+'K'):
                    caps += self._captures_from(r, c, self.board, [])
        if caps:
            return caps
        # simple moves
        moves = []
        for r in range(ROWS):
            for c in range(COLS):
                if self.board[r][c] in (player, player+'K'):
                    moves += self._simple_moves_from(r, c)
        return moves

    def _simple_moves_from(self, r, c):
        piece = self.board[r][c]
        dirs = DIRECTIONS[piece]
        out = []
        for dr, dc in dirs:
            nr, nc = r+dr, c+dc
            if self.inside(nr, nc) and self.board[nr][nc] == EMPTY:
                out.append(Move([(r, c), (nr, nc)]))
        return out

    def _captures_from(self, r, c, board, visited):
        piece = board[r][c]
        dirs = DIRECTIONS[piece]
        moves = []
        any_capture = False
        for dr, dc in dirs:
            mr, mc = r+dr, c+dc
            er, ec = r+2*dr, c+2*dc
            if self.inside(er, ec) and board[er][ec] == EMPTY:
                target = board[mr][mc]
                if target != EMPTY and target[0] != piece[0] and (mr,mc) not in visited:
                    any_capture = True
                    nb = copy.deepcopy(board)
                    nb[r][c] = EMPTY
                    nb[mr][mc] = EMPTY
                    nb[er][ec] = piece
                    #promote
                    if piece == WHITE and er == 0: nb[er][ec] = WHITE_KING
                    if piece == BLACK and er == 7: nb[er][ec] = BLACK_KING
                    next_moves = self._captures_from(er, ec, nb, visited+[(mr,mc)])
                    if next_moves:
                        for m in next_moves:
                            moves.append(Move([(r,c)]+m.path, [(mr,mc)]+m.captures))
                    else:
                        moves.append(Move([(r,c),(er,ec)], [(mr,mc)]))
        return moves

    def apply_move(self, move):
        b = self.board
        src, dst = move.src, move.dest
        piece = b[src[0]][src[1]]
        b[src[0]][src[1]] = EMPTY
        b[dst[0]][dst[1]] = piece
        for mr, mc in move.captures:
            b[mr][mc] = EMPTY
        #promote
        if piece == WHITE and dst[0] == 0:
            b[dst[0]][dst[1]] = WHITE_KING
        if piece == BLACK and dst[0] == 7:
            b[dst[0]][dst[1]] = BLACK_KING
        self.turn = BLACK if self.turn == WHITE else WHITE

    def evaluate(self):
        score = 0
        for r in range(ROWS):
            for c in range(COLS):
                p = self.board[r][c]
                if p == WHITE: score += 1+(7-r)*0.1
                if p == WHITE_KING: score += 2
                if p == BLACK: score -= 1+r*0.1
                if p == BLACK_KING: score -= 2
        return score

    def minimax(self, board_state, depth, alpha, beta, maximizing):
        if depth == 0 or not self.get_moves(self.turn if maximizing else self.turn):
            return self.evaluate(), None
        best_move = None
        if maximizing:
            value = -math.inf
            for m in self.get_moves(self.turn):
                saved = copy.deepcopy(self.board)
                saved_turn = self.turn
                self.apply_move(m)
                val, _ = self.minimax(self.board, depth-1, alpha, beta, False)
                if val > value:
                    value, best_move = val, m
                alpha = max(alpha, value)
                self.board, self.turn = saved, saved_turn
                if alpha >= beta:
                    break
            return value, best_move
        else:
            value = math.inf
            for m in self.get_moves(self.turn):
                saved = copy.deepcopy(self.board)
                saved_turn = self.turn
                self.apply_move(m)
                val, _ = self.minimax(self.board, depth-1, alpha, beta, True)
                if val < value:
                    value, best_move = val, m
                beta = min(beta, value)
                self.board, self.turn = saved, saved_turn
                if alpha >= beta:
                    break
            return value, best_move

    def ai_move(self):
        _, m = self.minimax(self.board, self.depth, -math.inf, math.inf, True)
        return m

    def is_over(self):
        return not self.get_moves(WHITE) or not self.get_moves(BLACK)

if __name__ == '__main__':
    game = CheckersGame()
    while not game.is_over():
        game.print_board()
        if game.turn == WHITE:
            moves = game.get_moves(WHITE)
            for i, m in enumerate(moves):
                print(f"{i}: {m}")
            try:
                idx = int(input("choose your move: "))
                if 0 <= idx < len(moves):
                    game.apply_move(moves[idx])
                else:
                    print("invalid move num.")
            except ValueError:
                print("please enter a valid num.")
        else:
            print("AI (BLACK) is thinking right now...")
            move = game.ai_move()
            if move:
                print(f"AI chose: {move}")
                game.apply_move(move)
            else:
                print("AI has no moves.")
    print("game over.")
    game.print_board()

  0 1 2 3 4 5 6 7
0  .  B  .  B  .  B  .  B
1  B  .  B  .  B  .  B  .
2  .  B  .  B  .  B  .  B
3  .  .  .  .  .  .  .  .
4  .  .  .  .  .  .  .  .
5  W  .  W  .  W  .  W  .
6  .  W  .  W  .  W  .  W
7  W  .  W  .  W  .  W  .

0: move((5, 0)->(4, 1))
1: move((5, 2)->(4, 1))
2: move((5, 2)->(4, 3))
3: move((5, 4)->(4, 3))
4: move((5, 4)->(4, 5))
5: move((5, 6)->(4, 5))
6: move((5, 6)->(4, 7))


KeyboardInterrupt: Interrupted by user

Task 2

In [ ]:
import math

class CardNode:
    def __init__(self, cards, turn_max, max_score=0, min_score=0):
        self.cards = cards            #remaining cards
        self.turn_max = turn_max      #true if max's turn, false if min's
        self.max_score = max_score    #accumulated max score
        self.min_score = min_score    #accumulated min score
        self.children = []            #child CardNode states
        self.minimax_value = None     #store evaluated max_score at root

class MinimaxAgent:
    def __init__(self, depth):
        self.depth = depth

    def formulate_goal(self, node):
        return "goal reached" if node.minimax_value is not None else "searching"

    def act(self, node, environment):
        status = self.formulate_goal(node)
        if status == "goal reached":
            return node.minimax_value
        return environment.compute_minimax(node, self.depth)

class Environment:
    def __init__(self):
        self.computed_states = []

    def compute_minimax(self, node, depth):
        if depth == 0 or not node.cards:
            node.minimax_value = node.max_score
            self.computed_states.append(node.cards)
            return node.max_score
        
        if node.turn_max:
            best = -math.inf
            for pick_idx in (0, -1):  #left or right
                pick = node.cards[pick_idx]
                rem = node.cards[1:] if pick_idx == 0 else node.cards[:-1]
                child = CardNode(rem,
                                 turn_max=False,
                                 max_score=node.max_score + pick,
                                 min_score=node.min_score)
                node.children.append(child)
                val = self.compute_minimax(child, depth - 1)
                best = max(best, val)
                if best >= math.inf: break
            
            node.minimax_value = best
            self.computed_states.append(node.cards)
            return best
        else:
            #min's greedy turn: pick smaller end
            left, right = node.cards[0], node.cards[-1]
            if left <= right:
                pick_idx = 0
            else:
                pick_idx = -1
            
            pick = node.cards[pick_idx]
            rem = node.cards[1:] if pick_idx == 0 else node.cards[:-1]
            child = CardNode(rem,
                             turn_max=True,
                             max_score=node.max_score,
                             min_score=node.min_score + pick)
            node.children.append(child)
            val = self.compute_minimax(child, depth - 1)
            node.minimax_value = val
            self.computed_states.append(node.cards)
            return val

def play_card_game(cards, depth=10):
    env = Environment()
    agent = MinimaxAgent(depth)
    root = CardNode(tuple(cards), turn_max=True)
    
    #run minimax
    best_score = agent.act(root, env)
    
    #play moves sequentially following the tree
    node = root
    max_score = 0
    min_score = 0
    print(f"initial cards: {list(node.cards)}\n")
    
    while node.cards:
        if node.turn_max:
            #pick child with node.minimax_value
            for child in node.children:
                if child.minimax_value == root.minimax_value:
                    pick = list(set(node.cards) - set(child.cards))[0]
                    max_score += pick
                    print(f"max picks {pick}, remaining cards: {list(child.cards)}")
                    node = child
                    break
        else:
            #single child in greedy
            child = node.children[0]
            pick = list(set(node.cards) - set(child.cards))[0]
            min_score += pick
            print(f"min picks {pick}, remaining cards: {list(child.cards)}")
            node = child
        root = node
    
    print(f"\nfinal scores - max: {max_score}, min: {min_score}")
    winner = "Max" if max_score > min_score else "Min" if min_score > max_score else "Draw"
    print(f"winner: {winner}")

#simulation example
play_card_game([4, 10, 6, 2, 9, 5], depth=10)


initial cards: [4, 10, 6, 2, 9, 5]

max picks 5, remaining cards: [4, 10, 6, 2, 9]
min picks 4, remaining cards: [10, 6, 2, 9]
max picks 10, remaining cards: [6, 2, 9]
min picks 6, remaining cards: [2, 9]
max picks 9, remaining cards: [2]
min picks 2, remaining cards: []

final scores - max: 24, min: 12
winner: Max


Task 3

In [ ]:
import random
import string

#constants
grid_size = 10
ship_lengths = [5, 4, 3, 3, 2]

def coord_to_index(coord):
    row = int(coord[1:]) - 1
    col = string.ascii_uppercase.index(coord[0].upper())
    return row, col

def index_to_coord(r, c):
    return f"{string.ascii_uppercase[c]}{r+1}"

def print_grid(grid, show_ships=False):
    header = "  " + " ".join(str(i + 1) for i in range(grid_size))
    print(header)
    for r in range(grid_size):
        line = []
        for c in range(grid_size):
            cell = grid[r][c]
            if cell == 'O':
                char = 'O' if show_ships else '~'
            elif cell == 'X':
                char = 'X'
            elif cell == 'M':
                char = 'M'
            else:
                char = '~'
            line.append(char)
        print(f"{string.ascii_uppercase[r]} " + " ".join(line))
    print()

class Board:
    def __init__(self):
        self.grid = [['~'] * grid_size for _ in range(grid_size)]
        self.ships = []

    def place_random_ships(self):
        for length in ship_lengths:
            placed = False
            while not placed:
                orient = random.choice(['H', 'V'])
                r = random.randint(0, grid_size - 1)
                c = random.randint(0, grid_size - 1)
                coords = [(r + (i if orient == 'V' else 0), c + (i if orient == 'H' else 0)) for i in range(length)]
                if all(0 <= rr < grid_size and 0 <= cc < grid_size and self.grid[rr][cc] == '~' for rr, cc in coords):
                    for rr, cc in coords:
                        self.grid[rr][cc] = 'O'
                    self.ships.append(set(coords))
                    placed = True

    def receive_attack(self, r, c):
        if self.grid[r][c] == 'O':
            self.grid[r][c] = 'X'
            for ship in self.ships:
                if (r, c) in ship:
                    ship.remove((r, c))
                    return 'sunk' if not ship else 'hit'
        elif self.grid[r][c] == '~':
            self.grid[r][c] = 'M'
            return 'miss'
        return 'already'

    def all_sunk(self):
        return all(not ship for ship in self.ships)

class BattleshipAI:
    def __init__(self):
        self.heatmap = [[0] * grid_size for _ in range(grid_size)]
        self.known_hits = []
        self.guessed = set()

    def update_heatmap(self, board):
        self.heatmap = [[0] * grid_size for _ in range(grid_size)]
        for length in ship_lengths:
            #horizontal
            for r in range(grid_size):
                for c in range(grid_size - length + 1):
                    if all(board.grid[r][c + i] in ('~', 'O') for i in range(length)):
                        for i in range(length):
                            self.heatmap[r][c + i] += 1
            #vertical
            for r in range(grid_size - length + 1):
                for c in range(grid_size):
                    if all(board.grid[r + i][c] in ('~', 'O') for i in range(length)):
                        for i in range(length):
                            self.heatmap[r + i][c] += 1

    def pick_move(self, board):
        if self.known_hits:
            r, c = self.known_hits[-1]
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < grid_size and 0 <= nc < grid_size and (nr, nc) not in self.guessed:
                    self.guessed.add((nr, nc))
                    return nr, nc

        self.update_heatmap(board)
        best_val = -1
        best = []
        for r in range(grid_size):
            for c in range(grid_size):
                if (r, c) not in self.guessed:
                    if self.heatmap[r][c] > best_val:
                        best = [(r, c)]
                        best_val = self.heatmap[r][c]
                    elif self.heatmap[r][c] == best_val:
                        best.append((r, c))
        choice = random.choice(best)
        self.guessed.add(choice)
        return choice

#setup
grid_player = Board()
grid_ai = Board()
grid_player.place_random_ships()
grid_ai.place_random_ships()
ai = BattleshipAI()

#game-loop
def game_loop():
    print("welcome to battleship!\nyour grid is hidden, just enter coordinates to attack (e.g., B4)")
    while True:
        print("your turn:")
        print_grid(grid_ai.grid, show_ships=False)
        move = input("enter coordinate to attack (e.g., B4):").strip().upper()
        try:
            r, c = coord_to_index(move)
        except:
            print("invalid")
            continue
        res = grid_ai.receive_attack(r, c)
        print(f"player attacks: {move} → {res}")
        if res in ('hit', 'sunk'):
            ai.known_hits.append((r, c))
        if grid_ai.all_sunk():
            print("you win!")
            break

        print("AI's turn:")
        ar, ac = ai.pick_move(grid_player)
        pos = index_to_coord(ar, ac)
        res = grid_player.receive_attack(ar, ac)
        print(f"AI attacks: {pos} → {res}")
        if res in ('hit', 'sunk'):
            ai.known_hits.append((ar, ac))

        print("your board:")
        print_grid(grid_player.grid, show_ships=True)
        if grid_player.all_sunk():
            print("AI wins!")
            break

if __name__ == '__main__':
    game_loop()

welcome to battleship!
your grid is hidden, just enter coordinates to attack (e.g., B4)
your turn:
  1 2 3 4 5 6 7 8 9 10
A ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
B ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
C ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
D ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
E ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
F ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
G ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
H ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
I ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
J ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

player attacks: B4 → miss
AI's turn:
AI attacks: E6 → miss
your board:
  1 2 3 4 5 6 7 8 9 10
A ~ ~ ~ ~ O ~ ~ ~ ~ ~
B ~ ~ ~ ~ O ~ ~ ~ ~ ~
C ~ ~ ~ ~ O ~ ~ ~ ~ ~
D O O O O O ~ ~ ~ ~ ~
E O ~ O ~ ~ ~ ~ ~ ~ ~
F O ~ O ~ M ~ ~ ~ ~ ~
G ~ ~ O ~ ~ ~ ~ ~ ~ ~
H ~ ~ O O O O ~ ~ ~ ~
I ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
J ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

your turn:
  1 2 3 4 5 6 7 8 9 10
A ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
B ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
C ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
D ~ M ~ ~ ~ ~ ~ ~ ~ ~
E ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
F ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
G ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
H ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
I ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
J ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

player attacks: B2 → miss
AI's turn:
AI attacks: F5 → miss
your board:
  1 2 3 4 5 6 7 